Self check to write a small delta table and then read it from a new spark session.

In [1]:
%load_ext autoreload
%autoreload 2
from importlib.metadata import version
result = {
    "area": "spark",
    "description": "Check that spark can use iceberg tables.",
    "passed": False,
    "message": "",
    "plugin": "spark",
    "freeds": version('freeds')
}

# %pip install freeds==0.2.22
# print(version('freeds'))

In [ ]:
import json

from freeds.spark import get_spark_session, show_spark_info
def clean_up(table, db):
    spark.sql(f"DROP TABLE IF EXISTS {table}")
    spark.sql(f"DROP DATABASE IF EXISTS {db} CASCADE")

try:
    spark = get_spark_session("self-check1")
    # cfg = spark.sparkContext.getConf().getAll()
    # cfg.sort()
    # for item in cfg:
    #     print(item)
    db_name = "freeds_cat.selfcheck_db"
    table_name = f"{db_name}.selfcheck_tbl"

    # create some data in a table
    print(f"Dropping and recreating database {db_name}")
    clean_up(table=table_name, db=db_name)
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    print(f"Writing table {table_name}")
    data = spark.range(100)
    (
        data.write.option(  # .mode("overwrite")  # Options: 'overwrite', 'append', 'ignore', 'error' (default)
            "mergeSchema", "true"
        )
        .format("iceberg")  # Options: 'parquet', 'csv', 'json', 'orc', 'iceberg', 'delta' etc.
        .saveAsTable(table_name)
    )
    spark.stop()

    # read the table we just created
    spark = get_spark_session("self-check2")
    data = spark.table(table_name)
    show_spark_info(spark)
    data.show(5)

    # clean up
    clean_up(table=table_name, db=db_name)
    spark.stop()

    result["message"] = "Executed spark cell ok."
    result["passed"] = True

except Exception as ex:
    result["message"] = result["freeds"] + ':' + str(ex)
    print(ex)
    result["passed"] = False

Running on freeds version: 0.2.22
Dropping and recreating database freeds_cat.selfcheck_db
Writing table freeds_cat.selfcheck_db.selfcheck_tbl


25/09/25 17:12:34 WARN JdbcCatalog: JDBC catalog is initialized without view support. To auto-migrate the database's schema and enable view support, set jdbc.schema-version=V1


Running on freeds version: 0.2.22


25/09/25 17:12:38 WARN JdbcCatalog: JDBC catalog is initialized without view support. To auto-migrate the database's schema and enable view support, set jdbc.schema-version=V1


==== spark app: self-check2 ====
Spark master: spark://spark-master:7077
Delta lake location: s3a://dwh/warehouse
S3 endpoint: http://s3-minio:9900


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows



In [3]:
print(json.dumps(result, indent=4))

{
    "area": "spark",
    "description": "Check that spark can use iceberg tables.",
    "passed": true,
    "message": "Executed spark cell ok.",
    "plugin": "spark",
    "freeds": "0.2.22"
}
